# ECO Data Processing

In [1]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.243:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 20g
--executor-memory 8g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=420000
--conf spark.shuffle.registration.timeout=42000
--conf spark.driver.cores=4
--conf spark.executor.cores=8
--conf spark.driver.maxResultSize=38
pyspark-shell
"""

import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="ECO")

env: SPARK_HOME=/opt/spark


In [2]:
sc._conf.set("spark.sql.execution.arrow.enabled", "true")
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
# sc._conf.set("spark.cassandra.auth.username", "mlprw")
# sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.set("spark.cassandra.connection.host", "10.47.65.95")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "Df^9c2%znt7S")

sc._conf.getAll()

[('spark.driver.cores', '4'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.cassandra.auth.password', 'Df^9c2%znt7S'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.app.name', 'ECO'),
 ('spark.driver.maxResultSize', '38'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:

In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.243:7077").\
    appName("ECO").\
    config(conf=sc._conf).\
    getOrCreate()
# spark

sqlContext = SQLContext(sc)
sqlContext

In [4]:
import datetime
import time
from calendar import monthrange

# def cc_month(cc_expiration_date):
#     cc_expiration_date = cc_expiration_date.replace('/', '')
#     expire_month = None
#     if len(cc_expiration_date) == 3:
#         expire_month = int(cc_expiration_date[:1])
#     else:    
#         expire_month = int(cc_expiration_date[:2])
                
#     return expire_month

def to_date(datestr):
    """Convert from date string to date"""
    try:
        if len(datestr) <= 10:
            struct = time.strptime(datestr, "%Y-%m-%d")
        else:
            struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
        date = datetime.date(struct.tm_year, struct.tm_mon, struct.tm_mday)
    except Exception as ex:
        raise ValueError("date '{}' does not match format 'yyyy-mm-dd'".format(datestr))
    return date

def to_date_cc_expire_date(datestr):
    datestr = str(datestr).replace('/','')
    struct = time.strptime(datestr, "%m%y")
    last_date = monthrange(struct.tm_year,struct.tm_mon)[1]
    date = datetime.date(struct.tm_year,struct.tm_mon, last_date)
    return date



def is_expired(row):
    try:
        cc_expired_date = to_date_cc_expire_date(row['cc_expiration_date'])
        txn_date = to_date(row['transaction_date_in_string'])
        return txn_date >= cc_expired_date
    except Exception as ex:
        print(ex)
        return False
    
def added_years(row):
    added_expiry_years = row['added_expiry_years']
    if added_expiry_years == 'ADD_YEARS':
        added_expiry_years = str(int(row['date_increment']))
        
    if isinstance(added_expiry_years, int):
        added_expiry_years = str(int(added_expiry_years))
    
    return added_expiry_years

In [10]:
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,...,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,failed_cc_expiration_date,eco_value
0,2018-04-14,avast,10002829801,6667293883,2018-04-14 01:37:30,0,NON3DS,La Banque Postale,RE,497040,...,0,2018-04-04 01:10:38,4,27051,Correct card number and re-try.[14] [] [111800...,correct_cc_retry,10,1923f614-12c8-42b8-b85c-189527513db5,0121,MINUS
1,2018-04-19,avast,10002829801,6685387493,2018-04-19 01:06:59,5,NON3DS,La Banque Postale,RE,497040,...,0,2018-04-04 01:10:38,4,27051,Correct card number and re-try.[14] [] [111800...,correct_cc_retry,15,1923f614-12c8-42b8-b85c-189527513db5,0121,0002
2,2018-04-14,avast,10002829801,6667293883,2018-04-14 01:37:28,0,NON3DS,La Banque Postale,RE,497040,...,0,2018-04-04 01:10:38,4,27051,Correct card number and re-try.[14] [] [111800...,correct_cc_retry,10,1923f614-12c8-42b8-b85c-189527513db5,0121,MINUS
3,2018-04-19,avast,10002829801,6685387493,2018-04-19 01:06:58,5,NON3DS,La Banque Postale,RE,497040,...,0,2018-04-04 01:10:38,4,27051,Correct card number and re-try.[14] [] [111800...,correct_cc_retry,15,1923f614-12c8-42b8-b85c-189527513db5,0121,0002
4,2018-04-14,avast,10003471801,6667540213,2018-04-14 02:03:31,0,NON3DS,CREDIT MUTUEL ARKEA,FR,513029,...,0,2018-04-04 03:03:44,4,27001,Transaction refused[05] [] [111800] [111800],txn_refused,10,7bcc287a-43ad-4467-8c21-d626864980b4,0620,MINUS
5,2018-04-19,avast,10003471801,6685936043,2018-04-19 02:25:05,4,NON3DS,CREDIT MUTUEL ARKEA,FR,513029,...,0,2018-04-04 03:03:44,4,27001,Transaction refused[05] [] [111800] [111800],txn_refused,15,7bcc287a-43ad-4467-8c21-d626864980b4,0620,0001
6,2018-04-14,avast,10003471801,6667540213,2018-04-14 02:03:30,0,NON3DS,CREDIT MUTUEL ARKEA,FR,513029,...,0,2018-04-04 03:03:44,4,27001,Transaction refused[05] [] [111800] [111800],txn_refused,10,7bcc287a-43ad-4467-8c21-d626864980b4,0620,MINUS
7,2018-04-19,avast,10003471801,6685936043,2018-04-19 02:25:03,4,NON3DS,CREDIT MUTUEL ARKEA,FR,513029,...,0,2018-04-04 03:03:44,4,27001,Transaction refused[05] [] [111800] [111800],txn_refused,15,7bcc287a-43ad-4467-8c21-d626864980b4,0620,0001
8,2018-04-20,defendde,10005651701,6688843363,2018-04-20 01:00:44,5,NON3DS,DEUTSCHER SPARKASSEN UND GIROVERBAND,DE,523253,...,0,2018-04-18 01:18:30,18,27050,Do not try again/use alternate payment card.[0...,Hard Fraud,2,be924ca4-a2df-4699-bbbe-c9f34e15bbbf,0921,0001
9,2018-04-27,defendde,10005651701,6713754803,2018-04-27 00:59:57,3,NON3DS,DEUTSCHER SPARKASSEN UND GIROVERBAND,DE,523253,...,0,2018-04-18 01:18:30,18,27050,Do not try again/use alternate payment card.[0...,Hard Fraud,9,be924ca4-a2df-4699-bbbe-c9f34e15bbbf,0921,MINUS


In [5]:
import time

import pandas as pd
import numpy as np

from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def cc_month(cc_expiration_date):
    cc_expiration_date = cc_expiration_date.replace('/', '')
    expire_month = None
    if len(cc_expiration_date) == 3:
        expire_month = int(cc_expiration_date[:1])
    elif len(cc_expiration_date) == 4:    
        expire_month = int(cc_expiration_date[:2])
                     
    return expire_month


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date


def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  
udf_cc_month = udf(cc_month, StringType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-02' ")

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-06-%' ")
df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-04-%' or received_date like '2019-05-%' ")
               
# .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
# or (added_expiry_years != '')

sdf =  df\
    .filter("(added_expiry_years is not null) and (added_expiry_years != '') ")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .withColumn("cc_month", udf_cc_month(col("cc_expiration_date")))\
    .cache()

print(sdf.schema)
    
    
# grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
#                             .add("failed_day_of_month", IntegerType())\
#                             .add("failed_response_code", StringType())\
#                             .add("failed_response_message", StringType())\
#                             .add("failed_decline_type", StringType())\
#                             .add("days_between", IntegerType())\
#                             .add("failed_event_uuid", StringType())\
#                             .add("failed_cc_expiration_date", StringType())\
#                             .add("eco_value", StringType())
                            
                        
# @pandas_udf( sdf.schema, PandasUDFType.GROUPED_MAP)
# def merge_by_sub_spark(spark_df):
#     pdf = pd.DataFrame(spark_df)
#     result = pdf.groupby(['subscription_id', 'subsegment_id', 'bank_code', 'card_brand', 'card_category', 'card_class']).apply(merge_by_eco_2)
#     return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())


# pdf_all = toPandas(sdf_filtered, 64)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape  # (8682889, 46)  

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(888489, 47)

In [1]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
raw_txn_2019_Q1 = pd.read_csv(WORK_DIR + 'raw_txns_2019_01_to_2019_04.csv')
raw_txn_2019_Q1.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,18,24,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(11542180, 56)

In [4]:
raw_txn_2019_05 = pd.read_csv(WORK_DIR + 'raw_txns_2019_05.csv')
raw_txn_2019_05.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,15,21,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2927429, 49)

In [ ]:
raw_txn_2019 = pd.concat([raw_txn_2019_Q1, raw_txn_2019_05], ignore_index=True)
raw_txn_2019.shape

In [5]:
raw_txn_2019_Q1.shape

(11542180, 56)

In [10]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all.head()

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
0,0,2018-07-25,kasperus,9171379701,7110047683,2018-07-25 07:16:11,NaN,A0FB8A10DDDE3D03,"PNC Bank, National Association",US,443040,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),1120,NaN,NaN,25,Wednesday,298d718f-01fc-4c9b-8c34-11ae085902a8,NaN,Debit,US,DRI*Kaspersky Lab,311009012882,DR_INC-ENTITY,Completed,DR_INC-ENTITY,105.99,105.990000,USD,Visa,firstdata,1.0,105.99,105.990000,USD,50256098100,100,00: Approved,New,11860413400,2018-07-25 02:16:11,2,Authorize,1.0
1,1,2018-07-25,adbehil,9325506901,7109605743,2018-07-25 06:06:02,NaN,NON3DS,Bank Leumi Le-Israel B.M.,IL,458012,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),0820,NaN,NaN,25,Wednesday,343906f5-cc7a-497c-a218-79d30b1dcb05,NaN,Credit,IL,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,70.00,19.186120,ILS,Visa,netgiro-seb,1.0,70.00,19.186120,ILS,50251259400,0,SUCCESS[00] [085586] [] [],New,16588643900,2018-07-25 01:06:02,1,Authorize,1.0
2,2,2018-07-25,avgstore,10424918001,7109803853,2018-07-25 06:41:53,NaN,NON3DS,Powszechna Kasa Oszczednosci Bank Polski SA (Pko Bank Polski,PL,416879,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1019,NaN,NaN,25,Wednesday,ca281481-73ab-44c6-b4cb-5a82fefe20e6,NaN,Debit,PL,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,219.99,59.640171,PLN,Visa,netgiro-seb,1.0,219.99,59.640171,PLN,50253521800,0,SUCCESS[00] [905414] [] [],New,13972225300,2018-07-25 01:41:53,1,Authorize,1.0
3,3,2018-07-25,avgstore,11936247401,7110351813,2018-07-25 08:46:04,3.0,28DF62E1FAF6EB0A,NaN,US,377227,American Express,NaN,NaN,True credit (No PIN/Signature capability),0920,NaN,3.0,25,Wednesday,531ca59d-cfeb-4778-8e51-bb6f22583502,3910821.0,NaN,NaN,DRI*AVG Technologies,941000108071_00000001,DR_INC-ENTITY,Declined,DR_INC-ENTITY,66.99,66.990000,USD,AmericanExpress,mes,4.0,66.99,66.990000,USD,50261112700,530,005: Do not honor,New,16671486200,2018-07-25 03:46:04,3,Authorize,0.0
4,4,2018-07-25,adbehap,3530818901,7109508933,2018-07-25 05:42:16,NaN,NON3DS,DBS Bank Ltd,SG,462845,Visa,Visa Platinum,Consumer,NON USA Consumer Debit - No Pin Access,0122,NaN,NaN,25,Wednesday,31e5c69f-1c66-4672-97ff-9661ac9deb7e,NaN,Debit,SG,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,20.00,14.662742,SGD,Visa,netgiro-seb,1.0,20.00,14.662742,SGD,50250313500,0,SUCCESS[00] [004657] [] [],New,16588252900,2018-07-25 00:42:16,0,Authorize,1.0


In [7]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
raw_txn_2019_Q1 = raw_txn_2019_Q1[~(raw_txn_2019_Q1['date_increment'].isna())]
raw_txn_2019_Q1.shape

(1553491, 56)

In [8]:
raw_txn_2019_Q1.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,decline_type,processor_att_num,processor_count,total_count,processor_count_by_att,total_count_by_att
14,14,14,14,14,2019-04-27,acd,9984290801,1000632107,2019-04-27 05:28:15,0,RB,BANQUE FEDERATIVE DU CREDIT MUTUEL (BFCM),FR,513283.0,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,1218,NaN,0.0,27,saturday,df124ebe-9690-45e8-942b-6f086bd501f5,2.84826e+06,Debit,FR,DRI*ACD Systems,1785390597-EUR-pacific,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,21.14,EUR,MasterCard,netgiro-bnp,3.0,18.99,21.14,EUR,60966167800,530,27001: Transaction refused,New,15905125400,2019-04-27 00:28:15,0,Authorize,0,txn_refused,1,16,27287,16,24250
15,15,15,15,15,2019-04-27,acd,9984290801,1000632427,2019-04-27 05:28:17,0,NON3DS,BANQUE FEDERATIVE DU CREDIT MUTUEL (BFCM),FR,513283.0,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,1218,RETRY_DECLINED.2,0.0,27,saturday,71aeaf00-558d-473f-bc23-fdc0b4c9233c,2.84826e+06,Debit,FR,DRI*ACD Systems,1411163460-EUR-pacific,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,21.14,EUR,MasterCard,netgiro-bms,3.0,18.99,21.14,EUR,60966167800,806,27050: Do not try again/use alternate payment card,New,15905125400,2019-04-27 00:28:17,0,Authorize,0,do_not_try,2,5702,27287,1725,3002
80,80,80,80,80,2019-04-27,adbehap,10005640501,1001063297,2019-04-27 09:00:56,0,4B5576EF9C6A1E0A,BANCO SANTANDER RIO SA,AR,544014.0,MasterCard,Gold MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),1118,NaN,0.0,27,saturday,f4c7d0f2-c2f7-4e69-aba8-12694a21cb4c,2.9541e+06,Credit,AR,DRI*Adobe Sales,DR_IRELAND_ROW,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,1596.00,36.50,ARS,MasterCard,adyen,2.0,1596.00,36.50,ARS,60982811800,522,05 : Do not honor,New,15934464900,2019-04-27 04:00:56,4,Authorize,0,do_not_honor,1,2374,4881,2374,3897
81,81,81,81,81,2019-04-27,adbehap,10005640501,1001063517,2019-04-27 09:00:57,0,NON3DS,BANCO SANTANDER RIO SA,AR,544014.0,MasterCard,Gold MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),1118,RETRY_DECLINED.2,0.0,27,saturday,950a7eeb-e7c0-4b2d-a5c6-07d8eed2ab0b,2.9541e+06,Credit,AR,DRI*Adobe Sales,1839936749-ARS-pacific,DR_IRELAND-ENTITY,Failed,DR_IRELAND-ENTITY,1596.00,36.50,ARS,MasterCard,drwp-dlocal-auto,2.0,1596.00,36.50,ARS,60982811800,225,38002: Illegal argument,Declined,15934464900,2019-04-27 04:00:57,4,Authorize,0,Base,2,190,4881,190,733
82,82,82,82,82,2019-04-27,adbehap,10005640501,1001063617,2019-04-27 09:00:59,0,NON3DS,BANCO SANTANDER RIO SA,AR,544014.0,MasterCard,Gold MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),1118,RETRY_FAILED.3,0.0,27,saturday,99d7be0c-35ed-4dd4-9597-d088056ba457,2.9541e+06,Credit,AR,DRI*Adobe Sales,1411163460-ARS-pacific,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,1596.00,36.50,ARS,MasterCard,netgiro-bms,2.0,1596.00,36.50,ARS,60982811800,530,27001: Transaction refused,New,15934464900,2019-04-27 04:00:59,4,Authorize,0,txn_refused,3,640,4881,190,251


In [14]:
raw_txn_2019_Q1[raw_txn_2019_Q1['bin'] == "443040"].groupby("bin").mean()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,subscription_id,transaction_id,day_of_month,payment_amount,payment_amount_usd,renew_att_num,request_amount,request_amount_usd,subsegment_id,transaction_hour,success,processor_att_num,processor_count,total_count,processor_count_by_att,total_count_by_att
bin,,,,,,,,,,,,,,,,,,,,
443040,5.928593e+06,5.928593e+06,5.928593e+06,5.928593e+06,8.335539e+09,8.301393e+09,15.216621,94.236567,81.770805,2.555858,94.236567,81.770805,1.529404e+10,8.294278,0.057221,1.415531,1078.463215,6895.694823,834.912807,4125.351499


In [12]:
FEATURES_FLOAT = ['date_increment', 'bin', 'added_expiry_years']
def handle_feat_float(df):
        for feat in FEATURES_FLOAT:
            df[feat] = df[feat].astype(str).str.replace('.0', '', regex=False)
        return df
    
raw_txn_2019_Q1 = handle_feat_float(raw_txn_2019_Q1)

In [32]:
pdf_all = pdf_all[~(pdf_all['date_increment'].isna())]

In [64]:
gp = raw_txn_2019_Q1.groupby(["bin", "date_increment"])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
date_increment_bin = df_gp[df_gp[('success', 'count')] >= 5]
date_increment_bin.head()

success       success_rate
                          sum count             
bin    date_increment                           
486330 3                   28    28        100.0
462419 3                   26    26        100.0
493861 0                   26    26        100.0
491858 3                   24    24        100.0
379855 3                   21    21        100.0

In [65]:
gp = raw_txn_2019_Q1.groupby(["bin", "added_expiry_years"])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
added_years_bin = df_gp[df_gp[('success', 'count')] >= 5]
added_years_bin.head()

success       success_rate
                              sum count             
bin    added_expiry_years                           
486330 3                       28    28        100.0
462419 3                       26    26        100.0
493861 0                       26    26        100.0
372784 4                       21    21        100.0
379855 3                       21    21        100.0

In [31]:
date_increment_bin

success       success_rate
                          sum count             
bin    date_increment                           
443040 4                    5    41    12.195122
       3                   28   458     6.113537
       1                    2    46     4.347826
       0                    7   189     3.703704

In [56]:
date_inc_list = range(0,6)
date_inc_names = ['date_inc_' + str(inc)  for inc in date_inc_list]

added_years = range(0,11)
added_years_names = ['add_year_' + str(inc)  for inc in added_years]

eco_bin = pd.DataFrame(columns=(date_inc_names + added_years_names))
eco_bin

,date_inc_0,date_inc_1,date_inc_2,date_inc_3,date_inc_4,date_inc_5,add_year_0,add_year_1,add_year_2,add_year_3,add_year_4,add_year_5,add_year_6,add_year_7,add_year_8,add_year_9,add_year_10


In [91]:
# eco_bin = pd.DataFrame(columns=(date_inc_names + added_years_names))
# eco_bin

,date_inc_0,date_inc_1,date_inc_2,date_inc_3,date_inc_4,date_inc_5,add_year_0,add_year_1,add_year_2,add_year_3,add_year_4,add_year_5,add_year_6,add_year_7,add_year_8,add_year_9,add_year_10


In [55]:
# added_years = range(0,11)
# added_years_names = ['add_year_' + str(inc)  for inc in added_years]
# added_years_names

['add_year_0',
 'add_year_1',
 'add_year_2',
 'add_year_3',
 'add_year_4',
 'add_year_5',
 'add_year_6',
 'add_year_7',
 'add_year_8',
 'add_year_9',
 'add_year_10']

In [66]:
date_increment_bin_pivot = date_increment_bin.reset_index().pivot(index='bin',columns='date_increment')[['success_rate']]
date_increment_bin_pivot.columns = date_increment_bin_pivot.columns.droplevel(0)
date_increment_bin_pivot.columns = ['date_inc_' + str(col[1])  for col in date_increment_bin_pivot.columns]
date_increment_bin_pivot.head()

,date_inc_0,date_inc_1,date_inc_2,date_inc_3,date_inc_4,date_inc_5
bin,,,,,,
370034,NaN,NaN,NaN,NaN,33.333333,NaN
370266,NaN,NaN,NaN,NaN,0.000000,NaN
370276,NaN,NaN,NaN,0.0,37.333333,0.0
370277,0.0,NaN,NaN,NaN,0.000000,NaN
370781,NaN,NaN,NaN,20.0,NaN,0.0


In [67]:
add_year_bin_pivot = added_years_bin.reset_index().pivot(index='bin',columns='added_expiry_years')[['success_rate']]
add_year_bin_pivot.columns = add_year_bin_pivot.columns.droplevel(0)
add_year_bin_pivot.columns = ['add_year_' + str(col[1])  for col in add_year_bin_pivot.columns]
add_year_bin_pivot.head()

,add_year_0,add_year_1,add_year_10,add_year_11,add_year_12,add_year_15,add_year_2,add_year_3,add_year_4,add_year_5,add_year_6,add_year_7,add_year_8,add_year_9
bin,,,,,,,,,,,,,,
370034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.333333,NaN,NaN,NaN,NaN,NaN
370266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
370276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.333333,0.0,NaN,NaN,NaN,NaN
370277,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
370781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [97]:
eco_bin = pd.concat([date_increment_bin_pivot, add_year_bin_pivot], axis=1,  join='inner')
eco_bin.shape

(12831, 20)

In [98]:
eco_bin.head()

,date_inc_0,date_inc_1,date_inc_2,date_inc_3,date_inc_4,date_inc_5,add_year_0,add_year_1,add_year_10,add_year_11,add_year_12,add_year_15,add_year_2,add_year_3,add_year_4,add_year_5,add_year_6,add_year_7,add_year_8,add_year_9
bin,,,,,,,,,,,,,,,,,,,,
370034,NaN,NaN,NaN,NaN,33.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.333333,NaN,NaN,NaN,NaN,NaN
370266,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
370276,NaN,NaN,NaN,0.0,37.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.333333,0.0,NaN,NaN,NaN,NaN
370277,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
370781,NaN,NaN,NaN,20.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [102]:
eco_bin.to_csv(WORK_DIR + 'eco_bin_profile.csv')

In [101]:
eco_bin.loc['601120']['add_year_3']

0.8928571428571428

In [21]:
def merge_by_bin(group):
    group["count"] = group.count()
    return group

In [ ]:
def merge_by_bin(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''
    

    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date', 'cid'], na_position='first')
    sorted_index = sorted_group.index.tolist()
    successful_df = sorted_group[sorted_group['success'] == 1]
    if not successful_df.empty:
        first_succes_index = successful_df.index[0]
        sorted_group_index = sorted_index[:sorted_index.index(first_succes_index) +1]
        sorted_group = sorted_group[sorted_group.index.isin(sorted_group_index)]
    
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = sorted_group[sorted_group['renew_att_num'] == str(next_att_num)].index.tolist()

#         initial_att_indexes = sorted_group[sorted_group['renew_att_num'] == (att_num)].index.tolist()
#         next_att_indexes = sorted_group[sorted_group['renew_att_num'] ==  (next_att_num)].index.tolist()
    
        if initial_att_indexes and next_att_indexes:
            last_initial_att_index = initial_att_indexes[-1]

            last_next_att_index = next_att_indexes[-1]        
    
            merged = sorted_group.loc[last_next_att_index].copy(deep=True)
            initial_row = sorted_group.loc[last_initial_att_index]    
            failed_attempt_date = initial_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = int(to_day(initial_row['transaction_date_in_string']))
            merged['day_of_month'] = int(to_day(merged['transaction_date_in_string']))
            merged['failed_response_code'] = initial_row['response_code']

            failed_response_msg = initial_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =  initial_row['event_uuid']  
            
###### Below are additional fields  ########
            merged['failed_cc_expiration_date'] = initial_row['cc_expiration_date']
#         merged['expiration_date_changed'] = str(initial_row['cc_expiration_date'] is not merged['cc_expiration_date'])

            initial_cc = initial_row['cc_expiration_date'].replace('/', '')
            merged_cc = merged['cc_expiration_date'].replace('/', '')
            merged['expiration_date_changed'] = (initial_cc != merged_cc)

            merged['processor_mid_changed'] = processor_mid_changed(initial_row, merged)
            
            merged['failed_cid'] = initial_row['cid']        
            merged['failed_payment_service_id'] = initial_row['payment_service_id']
            merged['failed_merchant_number'] = initial_row['merchant_number']
            
            merged['failed_bank_code'] = initial_row['bank_code']
            merged['bank_code_changed'] = merged['bank_code'] != merged['failed_bank_code']
            
            merged['failed_bin'] = initial_row['bin']
            merged['bin_changed'] = merged['bin'] != merged['failed_bin']
##############################           
            
            new_group = new_group.append(merged)    
            
             
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [ ]:
# Merge by bin to get the success rate of each date_increment
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta
import itertools

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date


    
udfValueToStatus = udf(convert_status, IntegerType())  
# udfValueToStatus = udf(convert_status, StringType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-04-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' or received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())\
                            .add("failed_cc_expiration_date", StringType())\
                            .add("expiration_date_changed", BooleanType())\
                            .add("processor_mid_changed", StringType())\
                            .add("failed_cid", StringType())\
                            .add("failed_payment_service_id", StringType())\
                            .add("failed_merchant_number", StringType())\
                            .add("failed_bank_code", StringType())\
                            .add("bank_code_changed", BooleanType())\
                            .add("failed_bin", StringType())\
                            .add("bin_changed", BooleanType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_next_calendar_attempt)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("bin")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 128)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape  # previous: (6608320, 53)

In [6]:
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,...,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,cc_month
0,2019-04-09,adbehap,10003870701,8639104243,2019-04-09 05:40:24,3,434BEBB2D6A182F8,Kasikornbank Public Company Limited,TH,494312,...,60241804700,530,Refused: Refused,New,18934521900,2019-04-09 00:40:24,00,Authorize,0,2
1,2019-04-09,adbehap,10003870701,8639104503,2019-04-09 05:40:25,3,NON3DS,Kasikornbank Public Company Limited,TH,494312,...,60241804700,530,27001: Transaction refused,New,18934521900,2019-04-09 00:40:25,00,Authorize,0,2
2,2019-04-09,adbehap,10004282501,8639099823,2019-04-09 05:40:05,3,NON3DS,HSBC France,HK,456122,...,60241756000,530,27001: Transaction refused,New,18934503800,2019-04-09 00:40:05,00,Authorize,0,2
3,2019-04-09,adbehap,10004282501,8639100053,2019-04-09 05:40:06,3,NON3DS,HSBC France,HK,456122,...,60241756000,530,27001: Transaction refused,New,18934503800,2019-04-09 00:40:06,00,Authorize,0,2
4,2019-04-09,adbehap,10004467401,8639919513,2019-04-09 09:00:31,3,890AB1B8DFC20C31,Prisma Medios de Pago S.A.,AR,402918,...,60256659300,201,14 : Invalid card number,New,18934206200,2019-04-09 04:00:31,04,Authorize,0,2
5,2019-04-09,adbehap,10022579401,8639922193,2019-04-09 09:00:42,3,01314ADFBB78055C,INDUSTRIAL AND COMMERCIAL BANK OF CHINA (ARGEN...,AR,548315,...,60256735300,201,14 : Invalid card number,New,19011161300,2019-04-09 04:00:42,04,Authorize,0,2
6,2019-04-09,adbehap,10040800601,8639104613,2019-04-09 05:40:25,3,C46A83AF04E429B3,SIAM COMMERCIAL BANK PUBLIC COMPANY LIMITED,TH,541029,...,60241797300,401,Blocked Card: Blocked Card,New,19059592100,2019-04-09 00:40:25,00,Authorize,0,2
7,2019-04-09,adbehap,10042150101,8640013333,2019-04-09 09:30:49,0,DDF226A3D4312F50,Prisma Medios de Pago S.A.,AR,450979,...,60260684500,522,Expired Card: Expired Card,New,19059646600,2019-04-09 04:30:49,04,Authorize,0,3
8,2019-04-09,adbehap,10042150101,8640013513,2019-04-09 09:30:50,0,NON3DS,Prisma Medios de Pago S.A.,AR,450979,...,60260684500,605,22053: Expired card,New,19059646600,2019-04-09 04:30:50,04,Authorize,0,3
9,2019-04-09,adbehap,10063911901,8639109073,2019-04-09 05:40:44,3,NON3DS,HANG SENG BANK LIMITED,HK,558427,...,60241838200,100,SUCCESS[00] [942812] [111800] [111800],New,19135345000,2019-04-09 00:40:44,00,Authorize,1,10


In [12]:
pdf_all.shape

(16050340, 47)

In [8]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,cc_month
0,2018-07-05,acd,10282812201,7010567623,2018-07-05 07:22:50,,NON3DS,ING Belgium SA/NV,BE,423608,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0120,,,5,thursday,2f3687dd-f0dc-43ab-bc26-c030e901698b,,Credit,BE,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,20.56,20.56,USD,Visa,netgiro-bms,1,20.56,20.56,USD,49625537800,0,SUCCESS[00] [533772] [111800] [111800],New,13731685000,2018-07-05 02:22:50,02,Authorize,1,1
1,2018-07-05,acd,10283211001,7010594363,2018-07-05 07:28:06,,NON3DS,NATIONAL WESTMINSTER BANK PLC,GB,552213,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0819,,,5,thursday,bfe370d4-c37b-40d6-9e90-6e9734d89926,,Credit,GB,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,20.39,20.39,USD,MasterCard,netgiro-bms,1,20.39,20.39,USD,49625815400,0,SUCCESS[00] [042258] [122800] [122800],New,13732093300,2018-07-05 02:28:06,02,Authorize,1,8
2,2018-07-05,acd,10283381301,7010595133,2018-07-05 07:28:14,,NON3DS,BANK OF THE PHILIPPINE ISLANDS,PH,542458,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),0320,,,5,thursday,958f0e8f-1061-44f3-b848-35d7663e32d4,,Credit,PH,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,16.99,16.99,USD,MasterCard,netgiro-bms,2,16.99,16.99,USD,49625826800,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],New,13732261200,2018-07-05 02:28:14,02,Authorize,0,3
3,2018-07-05,acd,10289267701,7010595233,2018-07-05 07:28:15,0,RB,Societe Generale S.A.,FR,415056,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1017,,0,5,thursday,35185729-c017-42b7-bb7b-936e5a41b85e,3788691,Debit,FR,DRI*ACD Systems,1785390597,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,22.108158,EUR,Visa,netgiro-bnp,1,18.99,22.108158,EUR,49625818400,27001,Transaction refused[05] [] [] [],New,13739217000,2018-07-05 02:28:15,02,Authorize,0,10
4,2018-07-05,acd,10289267701,7010595363,2018-07-05 07:28:17,0,NON3DS,Societe Generale S.A.,FR,415056,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1017,RETRY_DECLINED.2,0,5,thursday,9d6b1306-414f-4fc8-9f9b-d94b1daed59e,3788691,Debit,FR,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,22.108158,EUR,Visa,netgiro-bms,1,18.99,22.108158,EUR,49625818400,27001,Transaction refused[05] [] [111800] [111800],New,13739217000,2018-07-05 02:28:17,02,Authorize,0,10


In [13]:
pdf_all['added_expiry_years'].value_counts()

      12961261
3      1579394
0       605036
4       361684
5       294168
6       120640
2        76673
8        20199
1        11857
9         7271
10        6906
12        2765
7         2165
11         112
15          74
-3          63
-2          35
-1          23
-4          11
13           3
Name: added_expiry_years, dtype: int64

In [7]:
# pdf_all.to_csv(WORK_DIR + 'eco_2018_01_to_06.csv')
# pdf_all.to_csv(WORK_DIR + 'eco_2018_07_to_12.csv')
# pdf_all.to_csv(WORK_DIR + 'eco_2019_06.csv')
pdf_all.to_csv(WORK_DIR + 'eco_2019_04_to_05.csv')

In [10]:
bin_profile = pd.read_csv(WORK_DIR + 'bin_profile_2018_01_to_2019_03.csv')
bin_profile.shape

(38193, 13)

In [11]:
# FEATURES_FLOAT = ['bin', 'renew_att_num', 'failed_response_code']
FEATURES_FLOAT = ['bin']

def handle_feat_float(df):
        for feat in FEATURES_FLOAT:
            if feat in FEATURES_FLOAT:
                df[feat] = df[feat].apply(str).str.replace('.0', '', regex=False)
        return df

# handle_feat_float(eco)    
handle_feat_float(bin_profile)    

# retry_success.head()

,Unnamed: 0,Unnamed: 0.1,bin,Mean,Median,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,0,0,230540,23.170809,25.550000,6.990000,52.900000,12.315995,45.725000,50.275000,9.100000,9.100000,126
1,1,2,352811,37.533333,40.000000,32.000000,40.600000,4.801389,40.540000,40.588000,33.600000,33.600000,3
2,2,5,352815,172.055652,53.700000,18.000000,562.822610,261.654900,489.709218,548.199932,22.200000,22.200000,4
3,3,6,352818,15.643014,4.944068,4.800000,51.479167,16.386331,40.881000,49.359534,4.800000,4.800000,12
4,4,7,352820,39.909110,32.177332,30.700000,73.000000,16.556696,64.725000,71.345000,31.100000,31.100000,6
5,5,8,352821,49.015144,49.159598,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,48.115695,5
6,6,9,352822,35.688928,39.518769,18.714051,49.360468,9.785019,49.219043,49.332183,21.110764,21.110764,13
7,7,10,352823,41.450000,30.900000,30.900000,73.100000,21.100000,66.770000,71.834000,30.900000,30.900000,4
8,8,11,352824,56.229756,55.059513,40.000000,74.800000,14.297354,71.997854,74.239571,44.200000,44.200000,4
9,9,12,352830,38.264198,40.300000,30.800000,41.656793,5.019725,41.483274,41.622089,33.590000,33.590000,4


In [14]:
pdf_all = pd.merge(pdf_all, bin_profile[['bin','Mean', 'Median','Min','Max','StdDev','Max_95','Max_99','Min_1']], left_on='bin', right_on='bin', how='left')

pdf_all.shape  #10761324, 61   7343847, 63  (14522183, 55)

(16050340, 55)

In [15]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,cc_month,Mean,Median,Min,Max,StdDev,Max_95,Max_99,Min_1
0,2018-07-05,acd,10282812201,7010567623,2018-07-05 07:22:50,,NON3DS,ING Belgium SA/NV,BE,423608,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0120,,,5,thursday,2f3687dd-f0dc-43ab-bc26-c030e901698b,,Credit,BE,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,20.56,20.56,USD,Visa,netgiro-bms,1,20.56,20.56,USD,49625537800,0,SUCCESS[00] [533772] [111800] [111800],New,13731685000,2018-07-05 02:22:50,02,Authorize,1,1,55.682716,56.760000,2.49,907.452000,32.953988,98.203723,125.83808,14.396651
1,2018-07-05,acd,10283211001,7010594363,2018-07-05 07:28:06,,NON3DS,NATIONAL WESTMINSTER BANK PLC,GB,552213,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0819,,,5,thursday,bfe370d4-c37b-40d6-9e90-6e9734d89926,,Credit,GB,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,20.39,20.39,USD,MasterCard,netgiro-bms,1,20.39,20.39,USD,49625815400,0,SUCCESS[00] [042258] [122800] [122800],New,13732093300,2018-07-05 02:28:06,02,Authorize,1,8,60.579962,58.530000,2.49,554.655147,31.098654,104.942800,132.75000,26.726180
2,2018-07-05,acd,10283381301,7010595133,2018-07-05 07:28:14,,NON3DS,BANK OF THE PHILIPPINE ISLANDS,PH,542458,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),0320,,,5,thursday,958f0e8f-1061-44f3-b848-35d7663e32d4,,Credit,PH,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,16.99,16.99,USD,MasterCard,netgiro-bms,2,16.99,16.99,USD,49625826800,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],New,13732261200,2018-07-05 02:28:14,02,Authorize,0,3,19.390944,13.200000,2.49,570.600000,20.940768,47.696169,78.30000,9.326059
3,2018-07-05,acd,10289267701,7010595233,2018-07-05 07:28:15,0,RB,Societe Generale S.A.,FR,415056,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1017,,0,5,thursday,35185729-c017-42b7-bb7b-936e5a41b85e,3788691,Debit,FR,DRI*ACD Systems,1785390597,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,22.108158,EUR,Visa,netgiro-bnp,1,18.99,22.108158,EUR,49625818400,27001,Transaction refused[05] [] [] [],New,13739217000,2018-07-05 02:28:15,02,Authorize,0,10,51.544971,56.856817,2.99,124.627536,30.225956,92.929982,116.08689,2.990000
4,2018-07-05,acd,10289267701,7010595363,2018-07-05 07:28:17,0,NON3DS,Societe Generale S.A.,FR,415056,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1017,RETRY_DECLINED.2,0,5,thursday,9d6b1306-414f-4fc8-9f9b-d94b1daed59e,3788691,Debit,FR,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.99,22.108158,EUR,Visa,netgiro-bms,1,18.99,22.108158,EUR,49625818400,27001,Transaction refused[05] [] [111800] [111800],New,13739217000,2018-07-05 02:28:17,02,Authorize,0,10,51.544971,56.856817,2.99,124.627536,30.225956,92.929982,116.08689,2.990000


In [12]:
pdf_all['cc_month'] = pdf_all['cc_expiration_date'].apply(cc_month)

In [62]:
pdf_all['is_expired'] = pdf_all.apply(is_expired, axis=1)

In [74]:
pdf_all['added_expiry_years'] =  pdf_all.apply(added_years, axis=1)

In [37]:
pdf_all['is_expired'].value_counts(dropna=False)

False    1118919
True      404432
Name: is_expired, dtype: int64

In [73]:
pdf_all['added_years'].value_counts(dropna=False)

3        666443
0        403171
4        195838
5        116307
2         61920
6         41715
8         15513
1         12211
10         3324
9          3239
12         2026
7          1363
11          137
15          106
-2            8
-3            7
STALE         7
-1            6
-5            4
-4            3
13            3
Name: added_years, dtype: int64

In [75]:
pdf_all['added_expiry_years'].value_counts(dropna=False)

3        666443
0        403171
4        195838
5        116307
2         61920
6         41715
8         15513
1         12211
10         3324
9          3239
12         2026
7          1363
11          137
15          106
-2            8
-3            7
STALE         7
-1            6
-5            4
-4            3
13            3
Name: added_expiry_years, dtype: int64

In [66]:
pdf_all['added_expiry_years'].value_counts(dropna=False)

3            552324
0            331446
4            162677
3            114110
5             96455
0             71725
2             50961
6             35502
4             33156
5             19852
8             13153
2             10959
1             10193
6              6213
10             2922
9              2573
8              2360
1              2017
12             1565
7              1210
9               666
12              461
10              402
7               153
11              116
15               83
15               23
11               21
ADD_YEARS        15
STALE             7
-3                7
-2                5
-1                5
-5                4
-2                3
-4                3
13                3
-1                1
Name: added_expiry_years, dtype: int64

In [16]:
pdf_all['added_expiry_years'].value_counts(dropna=True)

             7159538
3             666434
0             403171
4             195833
5             116307
2              61920
6              41715
8              15513
1              12210
10              3324
9               3239
12              2026
7               1363
11               137
15               106
ADD_YEARS         15
-2                 8
-3                 7
STALE              7
-1                 6
-5                 4
-4                 3
13                 3
Name: added_expiry_years, dtype: int64

In [9]:
pdf_all['success'].value_counts(normalize=True)

0    0.860195
1    0.139805
Name: success, dtype: float64

In [61]:
pdf_all

,Unnamed: 0,Unnamed: 0.1,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,cc_month
0,0,0,2019-02-15,acd,2462462809,8297439003,2019-02-15 06:17:16,0,NON3DS,UBS Switzerland AG,LI,454713.0,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1217,NaN,0.0,15,friday,74919cdc-4f67-4a5c-8905-8f13fa4d6981,2.13696e+06,Credit,CH,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,14.99,16.89,EUR,Visa,netgiro-bms,3,14.99,16.89,EUR,57992095700,27001.0,Transaction refused[05] [] [111800] [111800],New,15251946800,2019-02-15 00:17:16,0,Authorize,0,12
1,1,1,2019-02-15,acd,2462462809,8297439053,2019-02-15 06:17:17,0,NON3DS,UBS Switzerland AG,LI,454713.0,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1217,RETRY_DECLINED.2,0.0,15,friday,5ad50927-dc3c-49be-8b6e-0fcd5e06827c,2.13696e+06,Credit,CH,DRI*ACD Systems,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,14.99,16.89,EUR,Visa,drwp-fd,3,14.99,16.89,EUR,57992095700,27001.0,Transaction refused[null] [null] [null] [null],New,15251946800,2019-02-15 00:17:17,0,Authorize,0,12
2,2,2,2019-02-15,acd,2463604509,8297439033,2019-02-15 06:17:16,3,NON3DS,"KEB Hana Card Co.,Ltd.",KR,411904.0,Visa,Visa Platinum,Consumer,Credit - True credit (No PIN/Signature capability),0122,NaN,NaN,15,friday,69e7d16b-7e89-45b4-9847-4bb33a005a0a,NaN,Credit,KR,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.69,18.69,USD,Visa,netgiro-bms,3,18.69,18.69,USD,57991656200,22053.0,Expired card.[54] [] [111800] [111800],New,15252060500,2019-02-15 00:17:16,0,Authorize,0,1
3,3,3,2019-02-15,acd,2463604509,8297439063,2019-02-15 06:17:18,3,NON3DS,"KEB Hana Card Co.,Ltd.",KR,411904.0,Visa,Visa Platinum,Consumer,Credit - True credit (No PIN/Signature capability),0122,RETRY_DECLINED.2,NaN,15,friday,2d1ee0f5-97c0-4531-8ed0-28aded5b3a4d,NaN,Credit,KR,DRI*ACD Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,18.69,18.69,USD,Visa,netgiro-seb,3,18.69,18.69,USD,57991656200,27001.0,Transaction refused[54] [000000] [] [],New,15252060500,2019-02-15 00:17:18,0,Authorize,0,1
4,4,4,2019-02-15,acd,2489810409,8297458043,2019-02-15 06:21:02,0,NON3DS,BAWAG P.S.K. BANK,AT,526624.0,MasterCard,MasterCard Credit Card (mixed BIN),Consumer,Credit - True credit (No PIN/Signature capability),0716,NaN,0.0,15,friday,5b0c7223-03d9-4c5b-806d-e0f86a24b469,1.9407e+06,Credit,AT,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,14.99,16.89,EUR,MasterCard,netgiro-bms,1,14.99,16.89,EUR,57992331400,0.0,SUCCESS[00] [526463] [111800] [111800],New,15383570200,2019-02-15 00:21:02,0,Authorize,1,7
5,5,5,2019-02-15,acd,2490453009,8297458393,2019-02-15 06:21:07,0,NON3DS,CORNER BANCA S.A.,CH,548659.0,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),1015,NaN,0.0,15,friday,4cd01247-1319-45a4-88f9-ea09bff2bb49,2.03374e+06,Credit,CH,DRI*ACD Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,22.00,24.80,EUR,MasterCard,netgiro-bms,1,22.00,24.80,EUR,57992331600,27001.0,Transaction refused[05] [] [111800] [111800],New,15383590900,2019-02-15 00:21:07,0,Authorize,0,10
6,6,6,2019-02-15,acd,2490453009,8297458613,2019-02-15 06:21:09,0,NON3DS,CORNER BANCA S.A.,CH,548659.0,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),

In [50]:
pdf_all[pdf_all['added_expiry_years'] == 'ADD_YEARS']['date_increment'].value_counts(dropna=False)

3.0    9
4.0    5
1.0    1
Name: date_increment, dtype: int64

In [55]:
pdf_all['added_expiry_years'].value_counts(dropna=False)

3            552324
0            331446
4            162677
3            114110
5             96455
0             71725
2             50961
6             35502
4             33156
5             19852
8             13153
2             10959
1             10193
6              6213
10             2922
9              2573
8              2360
1              2017
12             1565
7              1210
9               666
12              461
10              402
7               153
11              116
15               83
15               23
11               21
ADD_YEARS        15
STALE             7
-3                7
-2                5
-1                5
-5                4
-2                3
-4                3
13                3
-1                1
Name: added_expiry_years, dtype: int64

In [59]:
pdf_all['added_expiry_years'].value_counts(dropna=False)

NaN    1523336
3            9
4            5
1            1
Name: added_expiry_years, dtype: int64

In [53]:
pdf_all.shape

(1523351, 49)

In [58]:
# pdf_all = pdf_all.drop_duplicates(subset=['event_uuid', 'transaction_date_in_string'], keep='last')
# pdf_all.shape

# pdf_all['added_expiry_years']  = pdf_all[pdf_all['added_expiry_years'] == 'ADD_YEARS']['date_increment'].astype(int).astype(str)

In [52]:
pdf_all[pdf_all['added_expiry_years'] == 'ADD_YEARS'][['subscription_id', 'date_increment']]

,subscription_id,date_increment
415367,1647040009,3.0
738519,1988354309,3.0
895908,123457788,4.0
922920,3241313009,3.0
954667,11327955801,3.0
988680,237740508,4.0
1104916,9915382701,3.0
1120677,1268484309,3.0
1196505,9915382701,1.0
1212575,1288136309,4.0


In [74]:
pdf_all['success'].value_counts(normalize=True)

0    0.947745
1    0.052255
Name: success, dtype: float64

In [78]:
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,failed_cc_expiration_date,eco_value
0,2019-02-08,avast,10007475401,8258737583,2019-02-08 12:15:39,0,A327558B2E47F05F,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0418,,0,8,friday,efae77a9-9eba-47eb-8200-d3d235f2424a,1872543,Debit,FR,DRI*AVAST Software,DR_IRELAND,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,113.44,EUR,MasterCard,adyen,2,99.99,113.44,EUR,57730537600,522,05: Do not honor,New,13274386200,2019-02-08 06:15:39,06,Authorize,0,2019-02-03 08:38:19,3,530,Refused: Refused,txn_refused,5,d9683bf6-9d95-416e-9e46-a02a7b237bb0,0420,MINUS
1,2019-02-08,avast,10007475401,8258737583,2019-02-08 12:15:40,0,A327558B2E47F05F,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0418,,0,8,friday,efae77a9-9eba-47eb-8200-d3d235f2424a,1872543,Debit,FR,DRI*AVAST Software,DR_IRELAND,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,113.44,EUR,MasterCard,adyen,2,99.99,113.44,EUR,57730537600,522,05: Do not honor,New,13274386200,2019-02-08 06:15:39,06,Authorize,0,2019-02-03 08:38:19,3,530,Refused: Refused,txn_refused,5,d9683bf6-9d95-416e-9e46-a02a7b237bb0,0420,MINUS
2,2019-03-07,adbehbr,10029003801,8421456503,2019-03-07 06:41:51,,69E55200A86F9D64,NU PAGAMENTOS SA,BR,523421,MasterCard,Gold MasterCard Card,Consumer,Credit - Credit Hybrid (meaning it has pin capability also),0920,,,7,thursday,ec1d3e8f-84ee-491c-a8fb-ab20af12350d,,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,175.0,46.4,BRL,MasterCard,pagador,2,175.0,46.4,BRL,58876389100,530,7: Do Not Honor,New,18697071400,2019-03-07 00:41:51,00,Authorize,0,2019-02-28 00:58:26,28,530,7: Do Not Honor,do_not_honor,7,9e24f5ec-580e-457c-83d9-876164e78999,0920,
3,2019-02-09,adbehme,10041421401,8263669833,2019-02-09 06:38:24,0,NON3DS,NATIONAL BANK OF KUWAIT (S.A.K.),KW,541318,MasterCard,Titanium MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0418,,0,9,saturday,0edf73f5-8d62-4b5c-a28b-9481cda060e1,2100663,Credit,KW,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,2.8,9.2,KWD,MasterCard,netgiro-seb,2,2.8,9.2,KWD,57739299100,27001,Transaction refused[05] [000000] [] [],New,18403997400,2019-02-09 00:38:24,00,Authorize,0,2019-02-02 01:40:12,2,27001,Transaction refused[05] [000000] [] [],txn_refused,7,4ed793a8-2af3-4746-87b3-48b09221636f,0418,
4,2019-02-16,adbehme,10041421401,8303812223,2019-02-16 06:37:47,0,NON3DS,NATIONAL BANK OF KUWAIT (S.A.K.),KW,541318,MasterCard,Titanium MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0418,,0,16,saturday,b797d612-ee04-40c1-9294-ac2f156849a6,2100663,Credit,KW,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,2.8,9.2,KWD,MasterCard,netgiro-seb,3,2.8,9.2,KWD,58036325500,27001,Transaction refused[05] [000000] [] [],New,18403997400,2019-02-16 00:37:47,00,Authorize,0,2019-02-02 01:40:12,2,27001,Transaction refused[05] [000000] [] [],txn_refused,14,4ed793a8-2af3-4746-87b3-48b09221636f,0418,
5,2019-02-23,adbehme,100414

In [86]:
pdf_all['eco_value'].value_counts()

         2980109
MINUS     140219
0001       49466
0002       46173
0003       36114
0005       18748
0004       15716
0006        1958
0008         608
0010         564
0100         512
1101         310
NONE         309
0301         307
1100         299
0102         283
0201         279
0801         278
0101         277
0501         272
1002         270
0901         269
0402         262
1102         256
0200         246
1001         246
0601         242
0701         242
0401         241
0702         235
          ...   
0406          10
0206           8
0906           8
0506           7
0407           7
0806           7
0607           7
0107           7
0507           6
0908           6
0306           6
0510           5
0207           3
0907           3
0807           3
1008           2
0707           2
1223           2
0808           2
1106           2
0408           1
0307           1
0309           1
0209           1
0308           1
0409           1
0000           1
0108          

In [76]:
# pdf_all.to_csv(WORK_DIR + 'eco_2019_01_to_03.csv')
# pdf_all.to_csv(WORK_DIR + 'eco_new_2019_01_to_03.csv')
pdf_all.to_csv(WORK_DIR + 'eco_all_2019_01_to_03.csv')


In [60]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
pdf_all = pd.read_csv(WORK_DIR + 'eco_all_2019_01_to_03.csv')
pdf_all.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,22,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1523351, 49)

In [3]:
pdf_all['success'].value_counts(normalize=True)

0    0.860195
1    0.139805
Name: success, dtype: float64

In [15]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,days_between,failed_event_uuid,failed_cc_expiration_date,eco_value
0,2019-02-08,avast,10007475401,8258737583,2019-02-08 12:15:39,0,A327558B2E47F05F,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0418,,0,8,friday,efae77a9-9eba-47eb-8200-d3d235f2424a,1872543,Debit,FR,DRI*AVAST Software,DR_IRELAND,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,113.44,EUR,MasterCard,adyen,2,99.99,113.44,EUR,57730537600,522,05: Do not honor,New,13274386200,2019-02-08 06:15:39,06,Authorize,0,2019-02-03 08:38:19,3,530,Refused: Refused,5,d9683bf6-9d95-416e-9e46-a02a7b237bb0,,MINUS
1,2019-02-08,avast,10007475401,8258737853,2019-02-08 12:15:40,0,NON3DS,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0418,RETRY_DECLINED.2,0,8,friday,840199bd-00a6-426d-aa44-e4a155f48067,1872543,Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,113.44,EUR,MasterCard,netgiro-bms,2,99.99,113.44,EUR,57730537600,27050,Do not try again/use alternate payment card.[62] [] [111800] [111800],New,13274386200,2019-02-08 06:15:40,06,Authorize,0,2019-02-03 08:38:19,3,530,Refused: Refused,5,d9683bf6-9d95-416e-9e46-a02a7b237bb0,,MINUS
2,2019-02-02,adbehtw,10022801601,8224153893,2019-02-02 07:34:32,,AB132F8E1A6D64F9,"Chunghwa Post Company, Limited.",TW,470538,Visa,Visa Classic,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0126,,,2,saturday,9540c0b5-f3af-4996-a8d0-88a95918042f,,Debit,TW,DRI*Adobe Systems,311200593888,DR_TAIWAN-ENTITY,Completed,DR_TAIWAN-ENTITY,320.0,10.4,TWD,Visa,firstdata,2,320.0,10.4,TWD,57459017700,100,00: Approved,New,18336360100,2019-02-02 01:34:32,01,Authorize,1,2019-01-26 00:37:58,26,501,63: SERV NOT ALLOWED,7,36c12f31-f6eb-4dec-b71b-aaa807b41504,,0105
3,2019-03-14,adbehbr,10029003801,8467308633,2019-03-14 05:38:52,,1EA1B24E53466306,NU PAGAMENTOS SA,BR,522688,MasterCard,Platinum MasterCard Card,Consumer,Credit - Credit Hybrid (meaning it has pin capability also),0127,,,14,thursday,9d13faaf-c92c-4eb0-91cf-447de1443cc9,,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Completed,DR_BRAZIL-ENTITY,175.0,45.7,BRL,MasterCard,pagador,3,175.0,45.7,BRL,59170886500,100,0:Successful,New,18697071400,2019-03-14 00:38:52,00,Authorize,1,2019-02-28 00:58:26,28,530,7: Do Not Honor,14,9e24f5ec-580e-457c-83d9-876164e78999,,0407
4,2019-03-23,adbehbr,10096482201,8527994403,2019-03-23 05:39:47,,003F47ECE4256177,NU PAGAMENTOS SA,BR,523421,MasterCard,Gold MasterCard Card,Consumer,Credit - Credit Hybrid (meaning it has pin capability also),0227,,,23,saturday,94839f03-a4a1-4004-944f-05456ea5c50d,,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Completed,DR_BRAZIL-ENTITY,105.0,27.1,BRL,MasterCard,pagador,2,105.0,27.1,BRL,59543887300,100,0:Successful,New,18901954200,2019-03-23 00:39:47,00,Authorize,1,2019-03-16 00:37:18,16,530,7: Do Not Honor,7,098e3b22-de09-42ca-becd-7f45e6c5298b,,0206
5,2019-01-27,adbehtw,10114289701,8189500673,2019-01-27 06:35:05,,62C3DE9248B298FE,Citibank Taiwan Limited,TW,463670,Visa,Visa Platinum,Consumer,True c

In [4]:
pdf_all['added_expiry_years'].value_counts(dropna=False)

3            552324
0            331446
4            162677
3            114110
5             96455
0             71725
2             50961
6             35502
4             33156
5             19852
8             13153
2             10959
1             10193
6              6213
10             2922
9              2573
8              2360
1              2017
12             1565
7              1210
9               666
12              461
10              402
7               153
11              116
15               83
15               23
11               21
ADD_YEARS        15
STALE             7
-3                7
-2                5
-1                5
-5                4
-2                3
-4                3
13                3
-1                1
Name: added_expiry_years, dtype: int64

In [66]:
pdf_all[(pdf_all['eco_value'] == '0803') & (pdf_all['success'] == 1)]

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,failed_cc_expiration_date,eco_value
2769,2019-01-17,avast,3801031001,8131237193,2019-01-17 07:09:02,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457144,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0522,,,17,thursday,1e9e4b3f-64dd-473c-9def-df8ffb346a61,,Debit,DK,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,349,53.250998,DKK,Visa,netgiro-bms,3,349,53.250998,DKK,56803245600,0,SUCCESS[00] [080903] [111800] [111800],New,14998898500,2019-01-17 01:09:02,01,Authorize,1,2019-01-02 04:16:16,2,27001,Transaction refused[05] [] [111800] [111800],txn_refused,15,cb5cf01f-23c9-44de-a142-8103cd0c0ea5,0919,0803
2896,2019-03-15,avast,4395755201,8473967783,2019-03-15 05:30:33,,D4751E29E2C32573,FIRST HAWAIIAN BANK,GU,541528,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0722,,,15,friday,8dcbb222-ffa7-4f10-8ef0-0b63937dc6d2,,Credit,US,DRI*AVAST Software,941000108061_00000004,DR_INC-ENTITY,Completed,DR_IRELAND-ENTITY,59.99,59.99,USD,MasterCard,mes,2,59.99,59.99,USD,59212399500,100,SUCCESS Address Match,New,15655061600,2019-03-15 00:30:33,00,Authorize,1,2019-03-05 00:30:31,5,530,005: Do not honor,do_not_honor,10,fe66e6b6-ee9e-4690-bb7b-540706956da5,1119,0803
3637,2019-02-13,avast,12220156801,8286713383,2019-02-13 10:46:35,,NON3DS,DNB Bank ASA,NO,492556,Visa,Visa Classic,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0122,,,13,wednesday,50048b1c-8ee0-4df8-a171-c3cd0d5a9cad,,Debit,NO,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,1049.0,121.0,NOK,Visa,netgiro-bms,2,1049.0,121.0,NOK,57931809800,0,SUCCESS[00] [533203] [111800] [111800],New,17213465700,2019-02-13 04:46:35,04,Authorize,1,2019-02-03 08:02:08,3,27001,Transaction refused[05] [] [111200] [111200],txn_refused,10,3e6ccb84-3254-44ae-8ab9-194f1c67e3ed,0519,0803
7946,2019-01-16,avast,9679716601,8128950893,2019-01-16 20:05:34,,NON3DS,ANZ Bank New Zealand Limited,NZ,498872,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0422,,,16,wednesday,a0d85305-3c64-4040-a88c-340f102ff999,,Credit,NZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,120.74,82.313884,NZD,Visa,netgiro-bms,3,120.74,82.313884,NZD,56796699900,0,SUCCESS[00] [060125] [111800] [111800],New,12705752400,2019-01-16 14:05:34,14,Authorize,1,2019-01-01 00:32:56,1,27048,Lost or stolen card.[41] [] [111800] [111800],Hard Fraud,15,3ba527e5-2554-4769-94b6-2c68c20adbb5,0819,0803
14847,2019-03-12,avgstore,11440959901,8453495243,2019-03-12 06:02:18,,NON3DS,HSBC UK Bank plc,GB,454638,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),0322,,,12,tuesday,19da0a7d-88d1-423f-aba7-7add7f6db669,,Credit,GB,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,19.99,26.06,GBP,Visa,netgiro-bms,2,19.99,26.06,GBP,59089167100,0,SUCCESS[00] [009079] [144800] [144800],New,15732248700,2019-03-12 01:02:18,01,Authorize,1,2019-03-08 01:06:13,8,27001,Transaction refused[05] [] [144800] [144800],txn_refused,4,6d50b6c2-62e9-4859-b171-001d89b5cb4f,0719,0803
17810,2019-02-10,adbehkr,10259428101,8268929483,2019-02-10 06:38:46,,NON3DS,"

In [ ]:
pdf_all['cc_expiration_date'] 

In [6]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all[(pdf_all['added_expiry_years'] == '0') & (pdf_all['success'] == 0)]

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
409613,409613,2019-02-17,avast,4013690201,8310322483,2019-02-17 13:46:07,0,27FFA1D3156FC960,CREDIT AGRICOLE S.A.,FR,513771.0,MasterCard,Gold MasterCard Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,1118,NaN,0.0,17,sunday,b5a36f1f-387a-4410-b7cc-82c5c1b51e44,2583283,Debit,FR,DRI*AVAST Software,DR_IRELAND,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,69.99,78.810000,EUR,MasterCard,adyen,1,69.99,78.810000,EUR,58112915000,522.0,Expired Card: Expired Card,New,15403651500,2019-02-17 07:46:07,7,Authorize,0
409614,409614,2019-02-17,avast,4013690201,8310322703,2019-02-17 13:46:09,0,NON3DS,CREDIT AGRICOLE S.A.,FR,513771.0,MasterCard,Gold MasterCard Card Immediate Debit,Consumer,Debit - NON USA Consumer Debit - No Pin Access,1118,RETRY_DECLINED.2,0.0,17,sunday,de02b7c3-290a-46ef-b4ad-679d7829e9b4,2583283,Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,69.99,78.810000,EUR,MasterCard,netgiro-bms,1,69.99,78.810000,EUR,58112915000,27001.0,Transaction refused[05] [] [111800] [111800],New,15403651500,2019-02-17 07:46:09,7,Authorize,0
409648,409648,2019-02-17,avast,4013948801,8309925603,2019-02-17 10:45:37,0,866EEEE1643CACF0,Societe Generale S.A.,FR,456140.0,Visa,Visa Gold,Consumer,Credit - True credit (No PIN/Signature capability),0918,NaN,0.0,17,sunday,426aafb1-3a00-4676-9fd2-58ee305dfcf8,2497973,Deferred Debit,FR,DRI*AVAST Software,DR_IRELAND,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,49.99,56.290000,EUR,Visa,adyen,1,49.99,56.290000,EUR,58099349300,522.0,Expired Card: Expired Card,New,15402881400,2019-02-17 04:45:37,4,Authorize,0
409649,409649,2019-02-17,avast,4013948801,8309925833,2019-02-17 10:45:38,0,NON3DS,Societe Generale S.A.,FR,456140.0,Visa,Visa Gold,Consumer,Credit - True credit (No PIN/Signature capability),0918,RETRY_DECLINED.2,0.0,17,sunday,7280a3b9-27cc-4a36-b75d-2450277dacb4,2497973,Deferred Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,49.99,56.290000,EUR,Visa,netgiro-bms,1,49.99,56.290000,EUR,58099349300,27051.0,Correct card number and re-try.[14] [] [111800] [111800],New,15402881400,2019-02-17 04:45:38,4,Authorize,0
409675,409675,2019-02-17,avast,4014191701,8310322033,2019-02-17 13:46:06,0,03837DC8B3F9CAB9,America First Federal Credit Union,US,475824.0,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),1218,NaN,0.0,17,sunday,d68bc0c8-a792-4d1b-bf99-700977395f6e,2532163,Debit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.10,64.100000,USD,Visa,firstdata,1,64.10,64.100000,USD,58112903200,530.0,51: DECLINED,New,15402894900,2019-02-17 07:46:06,7,Authorize,0
409676,409676,2019-02-17,avast,4014191701,8310322133,2019-02-17 13:46:06,0,03837DC8B3F9CAB9,America First Federal Credit Union,US,475824.0,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),1218,RETRY_DECLINED.2,0.0,17,sunday,b8650ed3-3e86-4c0b-a67f-810b8f0bf401,2532163,Debit,US,DRI*AVAST Software,7236114,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.10,64.100000,USD,Visa,litle,1,64.10,64.100000,USD,58112903200,530.0,349: Do Not Honor,New,15402894900,2019-02-17 07:46:06,7,Authorize,0
409712,409712,2019-02-17,avast,4014412801,8308941903,2019-02-17 06:30:42,0,NON3DS,TSB Bank Plc,CN,476367.0,Visa,Visa Classic,Consumer,Debit - NON USA Consumer Debit - No Pi

In [9]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
pdf_all[(pdf_all['added_expiry_years'] == '-2') ]

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
751484,751484,2019-01-08,avgstore,12729267801,8076252533,2019-01-08 07:19:27,-2,76815F615E52E0C6,The Toronto-Dominion Bank,CA,452088.0,Visa,Visa Infinite,Consumer,True credit (No PIN/Signature capability),0213,NaN,0.0,8,tuesday,633a4bc6-29f7-4147-8a51-9316d5254028,1141743,Credit,CA,DRI*AVG Technologies,7319737,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,69.99,52.420362,CAD,Visa,litle,6,69.99,52.420362,CAD,56432305300,530.0,349: Do Not Honor,New,18186735700,2019-01-08 01:19:27,1,Authorize,0
751485,751485,2019-01-08,avgstore,12729267801,8076252613,2019-01-08 07:19:27,-2,NON3DS,The Toronto-Dominion Bank,CA,452088.0,Visa,Visa Infinite,Consumer,True credit (No PIN/Signature capability),0213,RETRY_DECLINED.2,0.0,8,tuesday,83a520ee-7bfd-41b6-a6fc-2beb6026e303,1141743,Credit,CA,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,69.99,52.420362,CAD,Visa,netgiro-bms,6,69.99,52.420362,CAD,56432305300,27001.0,Transaction refused[05] [] [111800] [111800],New,18186735700,2019-01-08 01:19:27,1,Authorize,0
753051,753051,2019-01-08,avgstore,12775621101,8076277923,2019-01-08 07:21:26,-2,NON3DS,BNP PARIBAS FORTIS,BE,544036.0,MasterCard,Gold MasterCard Card,Consumer,True credit (No PIN/Signature capability),1016,NaN,0.0,8,tuesday,71f43bd6-37ec-4a94-8460-56511ec9eeb0,1365733,Credit,BE,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,69.99,80.103555,EUR,MasterCard,netgiro-bms,2,69.99,80.103555,EUR,56432576700,27050.0,Do not try again/use alternate payment card.[04] [] [111800] [111800],New,18278330600,2019-01-08 01:21:26,1,Authorize,0


In [23]:
pdf_all[pdf_all['eco_value'] != 'MINUS' ]['success'].value_counts(normalize=True)

0    0.837527
1    0.162473
Name: success, dtype: float64

In [18]:
gp = pdf_all.groupby(['added_expiry_years', 'cc_month'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 1000]

success        success_rate
                                sum  count             
added_expiry_years cc_month                            
8                  2            515   1232    41.801948
                   4            436   1048    41.603053
                   5            512   1269    40.346730
                   8            411   1052    39.068441
                   9            415   1112    37.320144
                   3            446   1206    36.981758
                   6            445   1253    35.514765
                   11           407   1281    31.772053
4                  8           3309  11837    27.954718
                   7            661   2386    27.703269
                   8            683   2527    27.028097
                   7           3158  11771    26.828647
6                  9            805   3171    25.386313
4                  2           3477  13727    25.329642
6                  8            681   2740    24.854015
                   3            659   2663    24.746526
                   1            565   2306    24.501301
                   7            691   2831    24.408336
4                  2            686   2827    24.266006
6                  12           661   2724    24.265786
4                  3           3105  12989    23.904843
                   5            675   2879    23.445641
6                  6            754   3219    23.423423
4                  6           3151  13463    23.404887
                   3            548   2344    23.378840
                   4            606   2599    23.316660
                   11           713   3091    23.066969
6                  10           761   3303    23.039661
4                  5           3114  13558    22.967989
                   11          3675  16057    22.887214
...                             ...    ...          ...
5                  6            509   8435     6.034381
                   11           573   9582     5.979962
                   2            442   7437     5.943257
                   4            440   7588     5.798629
                   11           104   1876     5.543710
                   3             80   1449     5.521049
                   6             97   1785     5.434174
                   10           490   9397     5.214430
                   10           101   1975     5.113924
1                  12            48   1014     4.733728
5                  9            106   2451     4.324765
                   9            484  11778     4.109356
1                  11            43   1140     3.771930
                   10            38   1097     3.463993
2                  8             68   2160     3.148148
                   10            25   1119     2.234138
                   7             46   2080     2.211538
                   1             78   3876     2.012384
1                  9             22   1154     1.906412
2                  5             67   3777     1.773895
                   2             77   4458     1.727232
                   6             71   4167     1.703864
                   12            68   4091     1.662185
                   11            74   4647     1.592425
                   4             69   4374     1.577503
                   9            121   7923     1.527199
                   11            16   1051     1.522360
                   10            73   4852     1.504534
                   9             20   1600     1.250000
                   3             56   4556     1.229148

[135 rows x 3 columns]

In [68]:
gp = pdf_all.groupby(['added_expiry_years'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 1]

success        success_rate
                       sum  count             
added_expiry_years                            
                     13082  19952    65.567362
ADD_YEARS                1      2    50.000000
11                       6    113     5.309735
9                       11    357     3.081232
6                      141   4978     2.832463
3                     1485  55606     2.670575
0                     2145  85616     2.505373
7                       21    915     2.295082
12                       2     89     2.247191
4                     1139  57955     1.965318
2                      352  19419     1.812658
5                     1015  71504     1.419501
8                       22   1736     1.267281
10                      29   2295     1.263617
1                        3    887     0.338219
15                       0     97     0.000000
-2                       0      4     0.000000
13                       0      3     0.000000

In [17]:
gp = pdf_all.groupby(['added_expiry_years'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 1]

success          success_rate
                        sum    count             
added_expiry_years                               
ADD_YEARS                11       15    73.333333
12                     1188     2026    58.637710
                    3685503  7159538    51.476827
9                      1355     3239    41.833899
8                      6160    15513    39.708631
STALE                     2        7    28.571429
6                     10090    41715    24.187942
4                     44603   195833    22.776039
3                    100992   666434    15.154089
0                     39722   403171     9.852395
5                      7161   116307     6.156981
11                        7      137     5.109489
1                       470    12210     3.849304
10                      120     3324     3.610108
7                        34     1363     2.494497
2                      1057    61920     1.707041
15                        0      106     0.000000
-2                        0        8     0.000000
-3                        0        7     0.000000
-1                        0        6     0.000000
-5                        0        4     0.000000
-4                        0        3     0.000000
13                        0        3     0.000000

In [64]:
gp = pdf_all.groupby(['eco_value'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 100]

success         success_rate
              sum   count             
eco_value                             
0803          102     120    85.000000
0703          101     122    82.786885
0502          132     161    81.987578
0302          184     231    79.653680
0103          130     165    78.787879
0902          155     199    77.889447
0802          160     207    77.294686
1003          129     169    76.331361
0303          111     147    75.510204
0602          162     219    73.972603
0702          173     235    73.617021
1002          195     270    72.222222
1000          135     187    72.192513
0503          109     151    72.185430
1100          215     299    71.906355
0701          174     242    71.900826
0203          114     159    71.698113
0403           84     118    71.186441
0901          190     269    70.631970
1101          218     310    70.322581
1001          169     246    68.699187
0603           91     135    67.407407
0102          190     283    67.137809
0202          143     214    66.822430
1102          171     256    66.796875
0501          181     272    66.544118
0601          161     242    66.528926
0101          183     277    66.064982
0401          159     241    65.975104
0201          181     279    64.874552
0301          198     307    64.495114
0600          137     217    63.133641
0903          101     160    63.125000
0400          108     174    62.068966
0801          172     278    61.870504
0700           82     133    61.654135
0402          159     262    60.687023
0900          102     173    58.959538
1103          131     227    57.709251
0200          140     246    56.910569
1104           93     168    55.357143
0800           90     163    55.214724
0300          125     227    55.066079
0500           90     181    49.723757
0100          245     511    47.945205
1004           78     164    47.560976
0904           47     146    32.191781
0006          247    1960    12.602041
0003         2141   36123     5.926972
0005          848   18692     4.536700
0001         1942   49471     3.925532
0008           23     607     3.789127
0004          594   15731     3.775984
0002         1723   46182     3.730891
0009            5     138     3.623188
MINUS        4358  140238     3.107574
NONE            9     309     2.912621
0010           10     564     1.773050

In [69]:
sc.stop()